In [1]:
import logging
from Db import DbWriter, load_data
import pandas as pd
import time

In [ ]:
logging.basicConfig(filename='info.log', level=logging.INFO)
# NB this will run for ~8 hours
db = DbWriter("bolt://localhost:7687", "neo4j", "")

In [13]:
(train, test) = db.get_file_paths("active1000", 0.7)

In [17]:
df_test = load_data("active1000", test)
df_train = load_data("active1000", train)

In [4]:
df_test = df_test[df_test["url"] != "http://adressa.no"]
user_df = df_test.groupby(['userId']).size().reset_index(name='counts')
users = user_df["userId"].tolist()
len(users)

988

In [5]:
# Predict all users on popularity
start_time = time.time()
predictions_raw_df = db.predict_on_popularity(users, None)
predictions_raw_df.to_feather("predictions_raw_all_users_v2.feather")
logging.info(f"Prediction took: {((time.time() - start_time)/60.0)} minutes, or hours: {((time.time() - start_time)/3600.0)}")
# NB: Prediction took: 739.2234967947006 minutes, or hours: 12.320391614834467

In [12]:
# Recommender precision, 10 predictions per user, for all users, on popularity
predictions = pd.read_feather("predictions_raw_all_users.feather")
df_test_url = df_test[["userId", "url"]]
df_test_url = df_test_url[df_test_url["userId"].isin(predictions["userId"])]
df_test_url = df_test_url[df_test_url["url"].isin(predictions["url"])]
print(f"Predictions matching test data / all predictions: {len(df_test_url)/len(predictions)}")
# Predictions matching test data / all predictions: 0.006578947368421052

Predictions matching test data / all predictions: 0.006578947368421052


In [19]:
# Recommender recall, 10 predictions per user, for all users, on popularity
predictions = pd.read_feather("predictions_raw_all_users.feather")
df_train_url = df_train[["userId", "url"]]
df_train_url = df_train_url[df_train_url["userId"].isin(predictions["userId"])]
df_train_url = df_train_url[df_train_url["url"].isin(predictions["url"])]
print(f"Prediction matching train data: predictions in test data / train data: {len(predictions)/len(df_train_url)}")

Prediction matching train data: predictions in test data / train data: 1.6521739130434783
